# Machine learning using PySpark

In [1]:
import os
os.environ["JAVA_HOME"] = "C:\Java\jdk1.8.0_202"
os.environ["SPARK_HOME"] = "C:\spark"

In [2]:
import findspark
findspark.init() 

In [3]:
# Loading Package
import pyspark
from pyspark import SparkContext

In [4]:
sc = SparkContext(master='local[2]')

In [5]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [6]:
#creating spark session
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName("BDProject").getOrCreate()

# Workflow
- Data Prep
- Feature Engineering
- Build Model
- Evaluate

In [8]:
#Load Dataset
df = spark.read.csv('YouTube.csv',header=True, inferSchema=True)

In [9]:
#df.show()

In [10]:
#print(df.columns)

In [11]:
#df.printSchema()

In [12]:
#df.describe().show()

In [13]:
#df.groupBy('ratings_disabled').count().show()

In [14]:
df.show(1)

+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+
|               Title| ChannelTitle|CategoryID|                Tags|ViewCount| Likes|CommentCount|CommentsDisabled|RatingsDisabled|Year|Month|Day|Trending_Year|Trending_Month|Trending_Day|Trending_Days|   Label|
+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+
|I left youtube fo...|jacksepticeye|        24|jacksepticeye|fun...|  2038853|353790|       40228|               0|              0|2020|    8| 11|         2020|             8|          12|            1|trending|
+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+------

In [15]:
import pyspark.ml

In [16]:
#dir(pyspark.ml)

In [17]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [18]:
print(df.columns)

['Title', 'ChannelTitle', 'CategoryID', 'Tags', 'ViewCount', 'Likes', 'CommentCount', 'CommentsDisabled', 'RatingsDisabled', 'Year', 'Month', 'Day', 'Trending_Year', 'Trending_Month', 'Trending_Day', 'Trending_Days', 'Label']


In [19]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- ChannelTitle: string (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- CommentsDisabled: integer (nullable = true)
 |-- RatingsDisabled: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Trending_Year: integer (nullable = true)
 |-- Trending_Month: string (nullable = true)
 |-- Trending_Day: integer (nullable = true)
 |-- Trending_Days: integer (nullable = true)
 |-- Label: string (nullable = true)



In [20]:
df.dtypes

[('Title', 'string'),
 ('ChannelTitle', 'string'),
 ('CategoryID', 'int'),
 ('Tags', 'string'),
 ('ViewCount', 'int'),
 ('Likes', 'int'),
 ('CommentCount', 'int'),
 ('CommentsDisabled', 'int'),
 ('RatingsDisabled', 'int'),
 ('Year', 'int'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Trending_Year', 'int'),
 ('Trending_Month', 'string'),
 ('Trending_Day', 'int'),
 ('Trending_Days', 'int'),
 ('Label', 'string')]

In [21]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [22]:
from pyspark.sql.functions import count

display(df.groupBy("Trending_Year").agg(count("*")).select("Trending_Year","count(1)").toPandas())


,Trending_Year,count(1)
0,2023.0,1800
1,2022.0,72998
2,NaN,14
3,1.0,1
4,6.0,1
5,3.0,3
6,5.0,3
7,4.0,3
8,2020.0,27593
9,2.0,3


In [23]:
df.show(3)

+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+
|               Title| ChannelTitle|CategoryID|                Tags|ViewCount| Likes|CommentCount|CommentsDisabled|RatingsDisabled|Year|Month|Day|Trending_Year|Trending_Month|Trending_Day|Trending_Days|   Label|
+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+
|I left youtube fo...|jacksepticeye|        24|jacksepticeye|fun...|  2038853|353790|       40228|               0|              0|2020|    8| 11|         2020|             8|          12|            1|trending|
|TAXI CAB SLAYER K...|Eleanor Neale|        27|eleanor|neale|ele...|   236830| 16423|        1642|               0|              0|2020|    8| 11|      

In [24]:
#Label Encoding
LabelEncoding = StringIndexer(inputCol='Label', outputCol='Target').fit(df)
df = LabelEncoding.transform(df)

In [25]:
df.show(2)

+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+------+
|               Title| ChannelTitle|CategoryID|                Tags|ViewCount| Likes|CommentCount|CommentsDisabled|RatingsDisabled|Year|Month|Day|Trending_Year|Trending_Month|Trending_Day|Trending_Days|   Label|Target|
+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+------+
|I left youtube fo...|jacksepticeye|        24|jacksepticeye|fun...|  2038853|353790|       40228|               0|              0|2020|    8| 11|         2020|             8|          12|            1|trending|   0.0|
|TAXI CAB SLAYER K...|Eleanor Neale|        27|eleanor|neale|ele...|   236830| 16423|        1642|               0|         

In [26]:
#get the labels
LabelEncoding.labels

['trending', 'non-trending']

In [27]:
print(df.columns)

['Title', 'ChannelTitle', 'CategoryID', 'Tags', 'ViewCount', 'Likes', 'CommentCount', 'CommentsDisabled', 'RatingsDisabled', 'Year', 'Month', 'Day', 'Trending_Year', 'Trending_Month', 'Trending_Day', 'Trending_Days', 'Label', 'Target']


In [28]:
df2 = df.select('Label')

In [29]:
df2.printSchema()

root
 |-- Label: string (nullable = true)



In [30]:
#df2.toPandas().astype(float)

In [31]:
required_features = ['ViewCount', 'Likes', 'CommentCount']

In [32]:
#vector assembler
vec_assembler = VectorAssembler(inputCols=required_features, outputCol='features')

In [33]:
vec_df = vec_assembler.transform(df)

In [34]:
vec_df.show(3)

+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+------+--------------------+
|               Title| ChannelTitle|CategoryID|                Tags|ViewCount| Likes|CommentCount|CommentsDisabled|RatingsDisabled|Year|Month|Day|Trending_Year|Trending_Month|Trending_Day|Trending_Days|   Label|Target|            features|
+--------------------+-------------+----------+--------------------+---------+------+------------+----------------+---------------+----+-----+---+-------------+--------------+------------+-------------+--------+------+--------------------+
|I left youtube fo...|jacksepticeye|        24|jacksepticeye|fun...|  2038853|353790|       40228|               0|              0|2020|    8| 11|         2020|             8|          12|            1|trending|   0.0|[2038853.0,353790...|
|TAXI CAB SLAYER K...|Eleanor Neale|    

In [35]:
finalased_data = vec_df.select('features','Target')

In [36]:
finalased_data.show()

+--------------------+------+
|            features|Target|
+--------------------+------+
|[2038853.0,353790...|   0.0|
|[236830.0,16423.0...|   0.0|
|[2381688.0,146739...|   0.0|
|[613785.0,37567.0...|   0.0|
|[940036.0,87113.0...|   0.0|
|[1050143.0,89192....|   0.0|
|[1.1308046E7,6554...|   0.0|
|[1514614.0,156910...|   0.0|
|[277506.0,27420.0...|   0.0|
|[1123889.0,45803....|   0.0|
|[210345.0,12221.0...|   0.0|
|[122755.0,6072.0,...|   0.0|
|[876682.0,153585....|   0.0|
|[1213314.0,64247....|   0.0|
|[165064.0,3921.0,...|   0.0|
|[97824.0,2068.0,6...|   0.0|
|[9140911.0,296505...|   0.0|
|[115871.0,7599.0,...|   0.0|
|[149047.0,47715.0...|   0.0|
|[730082.0,40309.0...|   0.0|
+--------------------+------+
only showing top 20 rows



In [38]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalased_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='features', labelCol='Target')
regressor=regressor.fit(train_data)

Py4JJavaError: An error occurred while calling o178.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 12.0 failed 1 times, most recent failure: Lost task 1.0 in stage 12.0 (TID 216) (DESKTOP-TG9LCS8 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$3133/24915159: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.colle

# Train, Test split

In [ ]:
train_df,test_df = vec_df.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

In [ ]:
LR = LogisticRegression(featuresCol='features', labelCol='Target')

In [ ]:
LR_Model = LR.fit(train_df)